In [ ]:
import pandas as pd
import json
from google.colab import drive

from os import listdir
from os.path import isfile, join




# Extract the evidence sentences from the given Wikipedia pages

In [ ]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

onlyfiles = [f for f in listdir('/content/drive/MyDrive/wiki-pages/') if isfile(join('/content/drive/MyDrive//wiki-pages/', f))]

In [ ]:
evidences = dict()
evidences['id'] = list()
evidences['lines'] = list()
for f in onlyfiles:
    file  = '/content/drive/MyDrive/wiki-pages/'+f
    with open(file,'r') as f_:
        
        for l in f_:
            l_ = json.loads(l)
#             print(type(l))
#             print(type(l_))
            data = json.loads(l.rstrip("\n"))
            evidences['id'].append(data['id'])
            evidences['lines'].append(data["lines"].split('\n'))
              # break
            
  

In [ ]:
df_evidences = pd.DataFrame.from_records(evidences)

In [ ]:
df_evidences.head(5)

,id,lines
0,Snakebite_-LRB-album-RRB-,[0\tSnakebite is the first official release by...
1,Sin_Sukju,"[0\tSin Suk-ju -LRB- Korean : 신숙주 , hanja : 申叔..."
2,"South_Oroville,_California",[0\tSouth Oroville is a census-designated plac...
3,Southwest_Golf_Classic,[0\tThe Southwest Golf Classic was a PGA Tour ...
4,"St._Philip's_Cathedral,_San_Felipe",[0\tThe St. Philip 's Cathedral -LRB- Catedral...


In [ ]:
df_evidences[df_evidences['id'] == 'Sin_Sukju']

,id,lines
1,Sin_Sukju,"[0\tSin Suk-ju -LRB- Korean : 신숙주 , hanja : 申叔..."


In [ ]:
df_evidences.to_csv('evidences_from_wikipages.csv')

In [ ]:
!cp evidences_from_wikipages.csv "drive/My Drive/"

In [ ]:
train_data = pd.read_json('/content/drive/MyDrive/train.jsonl',lines = True)

# df_evidences = pd.read_csv('/content/drive/MyDrive/evidences_from_wikipages.csv')

In [ ]:
train_data.head(10)

,id,verifiable,label,claim,evidence
0,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [..."
1,150448,VERIFIABLE,SUPPORTS,Roman Atwood is a content creator.,"[[[174271, 187498, Roman_Atwood, 1]], [[174271..."
2,214861,VERIFIABLE,SUPPORTS,"History of art includes architecture, dance, s...","[[[255136, 254645, History_of_art, 2]]]"
3,156709,VERIFIABLE,REFUTES,Adrienne Bailon is an accountant.,"[[[180804, 193183, Adrienne_Bailon, 0]]]"
4,83235,NOT VERIFIABLE,NOT ENOUGH INFO,System of a Down briefly disbanded in limbo.,"[[[100277, None, None, None]]]"
5,129629,VERIFIABLE,SUPPORTS,Homeland is an American television spy thrille...,"[[[151831, 166598, Homeland_-LRB-TV_series-RRB..."
6,149579,NOT VERIFIABLE,NOT ENOUGH INFO,Beautiful reached number two on the Billboard ...,"[[[173384, None, None, None]]]"
7,229289,NOT VERIFIABLE,NOT ENOUGH INFO,Neal Schon was named in 1954.,"[[[273626, None, None, None]]]"
8,33078,VERIFIABLE,SUPPORTS,The Boston Celtics play their home games at TD...,"[[[49158, 58489, Boston_Celtics, 3]], [[49159,..."
9,6744,VERIFIABLE,SUPPORTS,The Ten Commandments is an epic film.,"[[[23513, 28977, The_Ten_Commandments_-LRB-195..."


In [ ]:
print('Training Samples: ',len(train_data))

Training Samples:  145449


# Things to do before submission:

- Basic data exploration
- Evidence sentence extraction from the wikipedia pages
- Based on claims and evidence, tell if the claim is verifiable
- Based on claim, evidence and verifiable, check the label
- Start with basic BiLSTM and GRU architectures

In [ ]:
print('Maximum length of the claim in the train data: ',max(len(d) for d in train_data.claim.values))

Maximum length of the claim in the train data:  614


In [ ]:
print('Minimum length of the claim in the train data: ',min(len(d) for d in train_data.claim.values))

Minimum length of the claim in the train data:  10


In [ ]:
print('Maximum length of the evidence in the train data: ',max(len(d) for d in train_data.evidence.values))

Maximum length of the evidence in the train data:  251


In [ ]:
print('Minimum length of the evidence in the train data: ',min(len(d) for d in train_data.evidence.values))

Minimum length of the evidence in the train data:  1


In [ ]:
train_data.label.value_counts()

SUPPORTS           80035
NOT ENOUGH INFO    35639
REFUTES            29775
Name: label, dtype: int64

In [ ]:
train_data.verifiable.value_counts()

VERIFIABLE        109810
NOT VERIFIABLE     35639
Name: verifiable, dtype: int64

In [ ]:
train_data.evidence.values[0]

[[[92206, 104971, 'Nikolaj_Coster-Waldau', 7],
  [92206, 104971, 'Fox_Broadcasting_Company', 0]]]

In [ ]:
train_evidence_id = train_data['evidence'].apply(lambda x: [k[2] for k in x[0]])

In [ ]:
train_evidence_sentence_id = train_data['evidence'].apply(lambda x: [k[3] for k in x[0]])

In [ ]:
from numpy import int64
train_data['evidence_id'] = train_evidence_id

# train_data['evidence_id'] = pd.to_numeric(train_data['evidence_id'])# .astype(df_evidences['id']) #= int64

# train_data.convert_objects(convert_numeric=True)

train_data['evidence_sentence_id'] = train_evidence_sentence_id

In [ ]:
train_data = train_data.explode(['evidence_id','evidence_sentence_id'])

In [ ]:
merged_train = pd.merge(train_data, df_evidences, how='left',left_on = 'evidence_id',right_on = 'id')

In [ ]:
merged_train.head(5)

,id_x,verifiable,label,claim,evidence,evidence_id,evidence_sentence_id,id_y,lines
0,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [...",Nikolaj_Coster-Waldau,7,Nikolaj_Coster-Waldau,[0\tNikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ...
1,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [...",Fox_Broadcasting_Company,0,Fox_Broadcasting_Company,[0\tThe Fox Broadcasting Company -LRB- often s...
2,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [...",Nikolaj_Coster-Waldau,7,Nikolaj_Coster-Waldau,[0\tNikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ...
3,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [...",Fox_Broadcasting_Company,0,Fox_Broadcasting_Company,[0\tThe Fox Broadcasting Company -LRB- often s...
4,150448,VERIFIABLE,SUPPORTS,Roman Atwood is a content creator.,"[[[174271, 187498, Roman_Atwood, 1]], [[174271...",Roman_Atwood,1,Roman_Atwood,"[0\tRoman Bernard Atwood -LRB- born May 28 , 1..."


In [ ]:
merged_train = merged_train.groupby('id_x').agg(list)
merged_train.dropna(inplace=True)

In [ ]:
merged_train.head(5)

,verifiable,label,claim,evidence,evidence_id,evidence_sentence_id,id_y,lines
id_x,,,,,,,,
3,[VERIFIABLE],[SUPPORTS],[Chris Hemsworth appeared in A Perfect Getaway.],"[[[[15732, 19585, 'Chris_Hemsworth', 2]]]]",[Chris_Hemsworth],[2],[Chris_Hemsworth],[[0\tChris Hemsworth -LRB- born 11 August 1983...
4,[NOT VERIFIABLE],[NOT ENOUGH INFO],[Chris Hemsworth disappeared in A Perfect Geta...,"[[[[15230, None, None, None]]]]",[None],[None],[nan],[nan]
7,[VERIFIABLE],[SUPPORTS],[Roald Dahl is a writer.],"[[[[15234, 18987, 'Roald_Dahl', 0]]]]",[Roald_Dahl],[0],[Roald_Dahl],[[0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _...
8,[VERIFIABLE],[REFUTES],[Roald Dahl is a governor.],"[[[[15233, 18986, 'Roald_Dahl', 0]]]]",[Roald_Dahl],[0],[Roald_Dahl],[[0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _...
9,[VERIFIABLE],[SUPPORTS],[Ireland has relatively low-lying mountains.],"[[[[15235, 18988, 'Ireland', 10]]]]",[Ireland],[10],[Ireland],[[0\tIreland -LRB- -LSB- ˈaɪərlənd -RSB- Éire ...


In [ ]:
final_train = merged_train

In [ ]:

final_train['verifiable'] = final_train['verifiable'].apply(lambda x: x[0])
final_train['claim'] = final_train['claim'].apply(lambda x: x[0])
final_train['evidence'] = final_train['evidence'].apply(lambda x: x[0])
final_train['evidence_id'] = final_train['evidence_id'].apply(lambda x: x[0])
final_train['evidence_sentence_id'] = final_train['evidence_sentence_id'].apply(lambda x: x[0] if x[0]!=None else x[0])
final_train['lines'] = final_train['lines'].apply(lambda x: x[0])
final_train['label'] = final_train['label'].apply(lambda x: x[0])
final_train.drop('id_y',axis =1,inplace = True)


 

final_train.head(5)

,verifiable,label,claim,evidence,evidence_id,evidence_sentence_id,lines
id_x,,,,,,,
3,VERIFIABLE,SUPPORTS,Chris Hemsworth appeared in A Perfect Getaway.,"[[[15732, 19585, Chris_Hemsworth, 2]]]",Chris_Hemsworth,2.0,[0\tChris Hemsworth -LRB- born 11 August 1983 ...
4,NOT VERIFIABLE,NOT ENOUGH INFO,Chris Hemsworth disappeared in A Perfect Getaway.,"[[[15230, None, None, None]]]",None,NaN,NaN
7,VERIFIABLE,SUPPORTS,Roald Dahl is a writer.,"[[[15234, 18987, Roald_Dahl, 0]]]",Roald_Dahl,0.0,[0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ...
8,VERIFIABLE,REFUTES,Roald Dahl is a governor.,"[[[15233, 18986, Roald_Dahl, 0]]]",Roald_Dahl,0.0,[0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ...
9,VERIFIABLE,SUPPORTS,Ireland has relatively low-lying mountains.,"[[[15235, 18988, Ireland, 10]]]",Ireland,10.0,[0\tIreland -LRB- -LSB- ˈaɪərlənd -RSB- Éire -...


In [ ]:
final_train.dropna(inplace = True)

In [ ]:
final_train['evidence_sentence_id'] = [int(x) for x in final_train['evidence_sentence_id'].values.tolist()]

In [ ]:
final_train.head(5)

,verifiable,label,claim,evidence,evidence_id,evidence_sentence_id,lines
id_x,,,,,,,
3,VERIFIABLE,SUPPORTS,Chris Hemsworth appeared in A Perfect Getaway.,"[[[15732, 19585, Chris_Hemsworth, 2]]]",Chris_Hemsworth,2,[0\tChris Hemsworth -LRB- born 11 August 1983 ...
7,VERIFIABLE,SUPPORTS,Roald Dahl is a writer.,"[[[15234, 18987, Roald_Dahl, 0]]]",Roald_Dahl,0,[0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ...
8,VERIFIABLE,REFUTES,Roald Dahl is a governor.,"[[[15233, 18986, Roald_Dahl, 0]]]",Roald_Dahl,0,[0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ...
9,VERIFIABLE,SUPPORTS,Ireland has relatively low-lying mountains.,"[[[15235, 18988, Ireland, 10]]]",Ireland,10,[0\tIreland -LRB- -LSB- ˈaɪərlənd -RSB- Éire -...
10,VERIFIABLE,REFUTES,Ireland does not have relatively low-lying mou...,"[[[15241, 18992, Ireland, 10]]]",Ireland,10,[0\tIreland -LRB- -LSB- ˈaɪərlənd -RSB- Éire -...


In [ ]:
final_train.to_csv('train_data_final.csv')

In [ ]:
!cp train_data_final.csv "drive/My Drive/"
